<a href="https://colab.research.google.com/github/Hsynyvz/deep-learning-keras-tf-tutorial/blob/master/Yolo_with_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive1


In [ ]:
!ls /mydrive

 ColabNotebooks		 images				    video
'Colab Notebooks'	 municipality_bus_utilization.csv   yolov4
'huseyin yavuz-cv.pdf'	'My Drive'


In [ ]:
!ls /mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV

'Yolo with OpenCV.ipynb'


In [ ]:
%cd /mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV

/content/drive/My Drive/ColabNotebooks/yolov4/Yolov3_with_OpenCV


In [ ]:
import cv2
import numpy as np
 
# Loading image
img = cv2.imread("/mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV/car.jpg")

In [ ]:
yolo_weight = "/mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV/yolov3.weights"
yolo_config = "/mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV/yolov3.cfg"
coco_labels = "/mydrive/ColabNotebooks/yolov4/Yolov3_with_OpenCV/coco.names"
net = cv2.dnn.readNet(yolo_weight, yolo_config)
 
# Load coco object names file
classes = []
with open(coco_labels, "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
# # Defining desired shape
fWidth = 320
fHeight = 320
 
# Resize image in opencv
img = cv2.resize(img, (fWidth, fHeight))
 
height, width, channels = img.shape

In [ ]:
# Convert image to Blob
blob = cv2.dnn.blobFromImage(img, 1/255, (fWidth, fHeight), (0, 0, 0), True, crop=False)
# Set input for YOLO object detection
net.setInput(blob)

In [ ]:
# Find names of all layers
layer_names = net.getLayerNames()
print(layer_names)
# Find names of three output layers
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(output_layers)

['conv_0', 'bn_0', 'relu_0', 'conv_1', 'bn_1', 'relu_1', 'conv_2', 'bn_2', 'relu_2', 'conv_3', 'bn_3', 'relu_3', 'shortcut_4', 'conv_5', 'bn_5', 'relu_5', 'conv_6', 'bn_6', 'relu_6', 'conv_7', 'bn_7', 'relu_7', 'shortcut_8', 'conv_9', 'bn_9', 'relu_9', 'conv_10', 'bn_10', 'relu_10', 'shortcut_11', 'conv_12', 'bn_12', 'relu_12', 'conv_13', 'bn_13', 'relu_13', 'conv_14', 'bn_14', 'relu_14', 'shortcut_15', 'conv_16', 'bn_16', 'relu_16', 'conv_17', 'bn_17', 'relu_17', 'shortcut_18', 'conv_19', 'bn_19', 'relu_19', 'conv_20', 'bn_20', 'relu_20', 'shortcut_21', 'conv_22', 'bn_22', 'relu_22', 'conv_23', 'bn_23', 'relu_23', 'shortcut_24', 'conv_25', 'bn_25', 'relu_25', 'conv_26', 'bn_26', 'relu_26', 'shortcut_27', 'conv_28', 'bn_28', 'relu_28', 'conv_29', 'bn_29', 'relu_29', 'shortcut_30', 'conv_31', 'bn_31', 'relu_31', 'conv_32', 'bn_32', 'relu_32', 'shortcut_33', 'conv_34', 'bn_34', 'relu_34', 'conv_35', 'bn_35', 'relu_35', 'shortcut_36', 'conv_37', 'bn_37', 'relu_37', 'conv_38', 'bn_38', 're

In [ ]:
outs = net.forward(output_layers)
print(outs[0].shape)
print(outs[1].shape)
print(outs[2].shape)

(300, 85)
(1200, 85)
(4800, 85)


In [ ]:
# Generating random color for all 80 classes
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
# Extract information on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        # Extract score value
        scores = detection[5:]
        # Object id
        class_id = np.argmax(scores)
        # Confidence score for each object ID
        confidence = scores[class_id]
        # if confidence > 0.5 and class_id == 0:
        if confidence > 0.5:
            # Extract values to draw bounding box
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
 
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [ ]:
# Draw bounding box with text for each object
font = cv2.FONT_HERSHEY_DUPLEX
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence_label = int(confidences[i] * 100)
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f'{label, confidence_label}', (x-25, y + 75), font, 1, color, 2)
 


In [ ]:
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()